In [1]:
# Michael Sarullo
# Science Fair 2021-2022

# Thank you to the Kaggle community for making this data available and this endeavor possible. :)

# Imports begin below
import sys
import os
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
import scipy as sp
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensorV2

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

RuntimeError: KeyboardInterrupt: 

In [ ]:
#config
class CFG:
    debug=False
    num_workers=16
    model_name='efficientnet_b3'
    size=1024
    batch_size=4
    seed=25
    target_size=2
    target_col='MGMT_value'
    n_fold=5
    trn_fold=[0,1,2,3,4]
    inference=True

In [ ]:
!pip install timm
import os

if not os.path.exists('./'):
    os.makedirs('./')
    
trainfp = 'rsna-miccai-brain-tumor-radiogenomic-classification/train'
testfp = 'rsna-miccai-brain-tumor-radiogenomic-classification/test'

In [ ]:
#pytorch seed/config for reproducing
def seed_torch(s = 25):
    random.seed(s)
    os.environ['PYTHONHASHSEED'] = str(s)
    np.random.seed(s)
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(s)
    torch.cuda.manual_seed(s)

seed_torch(seed=CFG.seed)

In [ ]:
test = os.listdir(testfp)
test = pd.DataFrame({'BraTS21ID' : test})
test['BraTS21ID'] = test['BraTS21ID'].astype(int)
test.head()

In [ ]:
#dataset class
class TestDataset(Dataset):
    #constructor
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['BraTS21ID'].values
        self.transform = transform
    #length getter
    def __len__(self):
        return len(self.df)
    #getter for specific image in directory - used for processing in neural net
    def __getitem__(self, idx):
        root = f'{test_filepath}/{str(self.file_names[idx]).zfill(5)}/'
        com = []
        for typ in ['FLAIR', 'T1w', 'T1wCE', 'T2w']: # diff types of MRI images in directory
            paths = os.listdir(root + typ)
            rnd = random.sample(paths, min(10, len(paths)))
            typ_imgs = []
            
            for f in rnd:
                file_path = f'{root}{typ}/{f}'
                dicom = pydicom.read_file(file_path)
                data = apply_voi_lut(dicom.pixel_array, dicom)
                if dicom.PhotometricInterpretation == "MONOCHROME1": #nearly all are monochrome
                    data = np.amax(data) - data
                data = data - np.min(data)
                data = data / np.max(data)
                image = (data * 255).astype(np.uint8)
                typ_imgs.append(cv2.resize(image, (CFG.size, CFG.size)))
            com.append(np.mean(typ_imgs, axis = 0))
        image = np.array(com).transpose((1,2,0)) / 255
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            image = image.float()
        return image

In [ ]:
# transforms to tensor for processing
def get_transforms(*, data):
    if data == 'valid':
        return A.Compose([
            ToTensorV2(),
        ])

In [ ]:
# actual neural net class declaration (efficientnet)
class CustomEfficientNet(nn.Module):
    #constructor
    def __init__(self, model_name=CFG.model_name, pretrained=False):
        super().__init__()
        self.conv = nn.Conv2d(4,3,1)
        self.model = timm.create_model(CFG.model_name, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, CFG.target_size)
    #convolutes image (moves forward in line)
    def forward(self, x):
        x = self.conv(x)
        x = self.model(x)
        return x

In [ ]:
#loads trained states
def load_state_eff(model_path):
    state_dict = torch.load(model_path)['model']
    return state_dict

#inference func makes preds (in the form of probabilities)
def inference(model_eff, states, test_loader, device):
    model_eff.to(device)
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        x=1
        for state in states:
            model_eff.load_state_dict(state)
            model_eff.eval()
            with torch.no_grad():
                y_preds = model_eff(images)
            avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
            x+=1
            
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs

In [ ]:
import timm
model_eff = CustomEfficientNet(CFG.model_name, pretrained=False) #model

# using 5 states pretrained from associated program (see other jupyter notebook in repo)
states = [
          load_state_eff('fold0.pth'),
          load_state_eff('fold1.pth'),
          load_state_eff('fold2.pth'),
          load_state_eff('fold3.pth'),
          load_state_eff('fold4.pth'),
    ]

In [ ]:
test_dataset = TestDataset(test, transform=get_transforms(data='valid'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True)

preds = inference(model_eff, states, test_loader, device)

# submission
test['MGMT_value'] = preds[:,1]
test[['BraTS21ID', 'MGMT_value']].to_csv('./final.csv', index=False)
test.head()

In [ ]:
predsReal = test.sort_values(by = 'BraTS21ID')
labels = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
predsReal

In [ ]:
labels

In [ ]:
binary = np.full(585, -1)

In [ ]:
totals = []
cutoff = 0
def check(predsReal, binary, totals, labels, cutoff):
    for row in range(0, len(predsReal)):
        if predsReal['MGMT_value'][row] > cutoff:
            binary[row] = 1
        else:
            binary[row] = 0
    diff = abs(labels['MGMT_value'] - binary)
    totals.append(diff.value_counts()[0])
        

In [ ]:
check(predsReal, binary, totals, labels, cutoff)

In [ ]:
# Accuracy for 0.55 cutoff (182/256 = 71.09%)
# z = 
# error = 
high = [x for x in range(0,len(predsReal)) if list(predsReal['MGMT_value'])[x] > 0.55 or list(predsReal['MGMT_value'])[x] < 0.35]
highLabels = labels['MGMT_value'][high]
highReals = binary[high]
realDiff = abs(highLabels - highReals)
print(realDiff.value_counts(normalize = True))
print(len(realDiff))
print(realDiff.value_counts(normalize = True).iloc[0])

In [ ]:
# Accuracy for 0.6 cutoff (125/167 = 74.85%)
# z = 
# error = 
high = [x for x in range(0,len(predsReal)) if list(predsReal['MGMT_value'])[x] > 0.6 or list(predsReal['MGMT_value'])[x] < 0.35]
highLabels = labels['MGMT_value'][high]
highReals = binary[high]
realDiff = abs(highLabels - highReals)
print(realDiff.value_counts(normalize = True))
print(len(realDiff))
print(realDiff.value_counts(normalize = True).iloc[0])

In [ ]:
# Accuracy for 0.65 cutoff (54/70 = 77.14%)
# z = 
# error = 
high = [x for x in range(0,len(predsReal)) if list(predsReal['MGMT_value'])[x] > 0.65 or list(predsReal['MGMT_value'])[x] < 0.35]
highLabels = labels['MGMT_value'][high]
highReals = binary[high]
realDiff = abs(highLabels - highReals)
print(realDiff.value_counts(normalize = True))
print(len(realDiff))
print(realDiff.value_counts(normalize = True).iloc[0])

In [ ]:
# Accuracy for 0.7 cutoff (12/13 = 92.31%)
# z = 
# error = 
high = [x for x in range(0,len(predsReal)) if list(predsReal['MGMT_value'])[x] > 0.7 or list(predsReal['MGMT_value'])[x] < 0.3]
highLabels = labels['MGMT_value'][high]
highReals = binary[high]
realDiff = abs(highLabels - highReals)
print(realDiff.value_counts(normalize = True))
print(len(realDiff))
print(realDiff.value_counts(normalize = True).iloc[0])

In [ ]:
# Accuracy for 0.66 cutoff (54/70 = 83.05%)
# z = 
high = [x for x in range(0,len(predsReal)) if list(predsReal['MGMT_value'])[x] > 0.66 or list(predsReal['MGMT_value'])[x] < (1-.66)]
highLabels = labels['MGMT_value'][high]
highReals = binary[high]
realDiff = abs(highLabels - highReals)
print(realDiff.value_counts(normalize = True))
print(len(realDiff))
print(realDiff.value_counts(normalize = True).iloc[0])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
cuts = []
nums = []
accs = [0.5247863247863248,0.53125,0.5357142857142857,0.5301724137931034,0.5213270142180095,0.5408970976253298,0.5459770114942529,0.559375,0.5714285714285714,0.6016597510373444,0.6323529411764706,0.6464088397790055,0.6689655172413793,0.6949152542372882,0.7391304347826086,0.7714285714285715,0.84,0.8048780487804879,0.8571428571428571,0.8888888888888888,0.9230769230769231,0.8888888888888888,1.0,1.0, 1.0, 1.0]
def checkAcc(predsReal, labels, binary, cut):
    high = [x for x in range(0,len(predsReal)) if list(predsReal['MGMT_value'])[x] > cut or list(predsReal['MGMT_value'])[x] < (1-cut)]
    highLabels = labels['MGMT_value'][high]
    highReals = binary[high]
    realDiff = abs(highLabels - highReals)
    cuts.append(cut)
    nums.append(len(realDiff))
    #print(realDiff.value_counts(normalize = True).iloc[0])
    

    
for p in [0.5,0.51,0.52,0.53,0.54,0.55,.56,.57,.58,.59,.6,.61,.62,.63,.64,.65,.66,.67,.68,.69,.7,.71,.72,.73,.74,.75]:
    checkAcc(predsReal, labels, binary, p)
    
print(len(cuts))
print(len(accs))
print(len(nums))
q = sns.regplot(cuts,accs, color="#980000",scatter_kws={'s': nums})
q.set_xlabel("Cutoff Value", fontsize = 10)
q.set_ylabel("Model Accuracy", fontsize = 10)